# Simulation of Mass Action Kinetics
This is an example notebook for interactive simulation of a user-defined chemical reaction network. 

## Initializing a Chemical Reaction Network
Below is an example enzymatic reaction modelled with mass action kinetics. Refer to the README for more detailed instruction on defining a chemical reaction network.

In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
from kinetics_simulator.guis import ProgressCurveGUI, Slider
from kinetics_simulator.chemicalkinetics import ChemicalReactionNetwork

# define a reaction dictionary for reactions modeled with mass action kinetics
# a reversible reaction may be defined with a "<->" character
# the forward rate constant of a given reaction must be provided first in the ordered dictionary
reaction_dict = {
    'S + E <-> SE': {'model': 'mass-action', 'rate-constants': [100 ,1], 'rate-constant-names': ['kon', 'koff']}, # binding of substrate to enzyme
    'SE -> P + E': {'model': 'mass-action', 'rate-constants': 100, 'rate-constant-names': 'kchem'} # chemical step
}

# define an initial state for the system
# if an initial value for a specie is not given, the code assigns it as zero
initial_values = {'S': 1000, 'E': 0.1}
reaction_network = ChemicalReactionNetwork(initial_values, reaction_dict, time=np.linspace(0,1000,1000))

# define a set of sliders for rate constants and specie concentrations
# see the definition of the Slider class in ./kinetics_simulator/guis.py
# for a description of args and kwargs
sliders = [
    Slider('kon', min=-6, max=6, stepsize=1), 
    Slider('koff', min=-6, max=6, stepsize=1),
    Slider('kchem', min=-6, max=6, stepsize=1),
    Slider('S', min=-6, max=6, stepsize=1),
    Slider('E', min=-6, max=6, stepsize=1),
    ]

# set multithread to True for parallel computing
figure = ProgressCurveGUI(reaction_network, sliders=sliders, multithread=True, title='Michaelis Menten Kinetic Mechanism')
figure.interactive()

    'data': [{'name': 'S',
              'type': 'scatter',
              'uid':…

## Fitting a Simple Kinetic Model
Still in beta.

In [10]:
import numpy as np

reaction_dictionary = {
    'A + B <-> C': {'model': 'mass-action', 'rate-constants': [0.001,0.002], 'rate-constant-names': ['k1', 'k-1']},
}
plot_kwargs = {'title': 'Test',
                'fontsize': 12,
                'figsize': (12,12),
                'multithread': True, # turn on multithreading to leverage parallel processing
                'tol': 1e-8, # tolerance for the numerical integrator
                'A': {'min': 0, 'max': 1000, 'start': 100, 'stepsize': 1}
                }

initial_values = {'A': 100, 'B': 1000}
rxn = ChemicalReactionNetwork(initial_values, reaction_dictionary, time=np.linspace(0,30,500))
# result = rxn.fit(variable='A', observable='C', params=['k1', 'k-1'], fitting_concentrations=np.array([10, 50, 100, 500, 1000]))

# # the value of k1 and k-1 respectively
# print(result.x)

## Fitting a Kinetic Model with Input Data
Still in beta.

In [7]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../')
from kinetics_simulator.guis import ProgressCurveGUI
from kinetics_simulator.chemicalkinetics import ChemicalReactionNetwork

reaction_dictionary = {
    'PNP + Pi <-> PNP:Pi': {'model': 'mass-action', 'rate-constants': [0.001,0.002], 'rate-constant-names': ['kon', 'koff']},
    'PNP:Pi -> PNP + X': {'model': 'mass-action', 'rate-constants': 0.002, 'rate-constant-names': 'kcat'}
}

data = pd.read_csv('./PNP_data.csv')
Pi_concens = [float(concen.split(' ')[0]) for concen in data.columns[1:-1]]
time = data['time'].to_numpy()
timecourse = data.iloc[:, 1:-1].to_numpy().T


initial_values = {'PNP': 10e-3}
rxn = ChemicalReactionNetwork(initial_values, reaction_dictionary, time=time)
result = rxn.fit(variable='Pi', observable='X', params=['kon', 'koff', 'kcat'], fitting_concentrations=Pi_concens, ground_truth_data=timecourse)

# # the value of k1 and k-1 respectively
# print(result.x)